In [1]:
import numpy as np
import matplotlib.pyplot as plt
from matplotlib import cm
from ipywidgets import interactive, FloatSlider, IntSlider
from mpl_toolkits.mplot3d import Axes3D
from perlin_noise import PerlinNoise
import pylab as plt
from pythonworley import worley

/Users/aleksandrazinoveva/N3XTCODER/Civic Coding/07_Voice_to_Form/.venv/lib/python3.12/site-packages/pythonworley/__init__.py:4: UserWarning: pkg_resources is deprecated as an API. See https://setuptools.pypa.io/en/latest/pkg_resources.html. The pkg_resources package is slated for removal as early as 2025-11-30. Refrain from using this package or pin to Setuptools<81.
  from pkg_resources import get_distribution


In [2]:
def generate_base_torus(R=3, r=1, n_major=100, n_minor=30):
    """
    Generate points on a smooth torus surface.
    R: major radius (from center of hole to center of tube)
    r: minor radius (radius of tube)
    n_major: number of divisions along major circumference
    n_minor: number of divisions along minor circumference (cross section)
    Returns (X, Y, Z, normals, u, v):
      X, Y, Z: 3D coordinates of surface points (n_minor x n_major)
      normals: surface normal vectors (n_minor x n_major x 3)
      u: angles along major ring (0 to 2pi)
      v: angles along minor ring (0 to 2pi)
    """
    u = np.linspace(0, 2*np.pi, n_major, endpoint=True)
    v = np.linspace(0, 2*np.pi, n_minor, endpoint=True)
    U, V = np.meshgrid(u, v)

    # Parametric equations of torus
    X = (R + r * np.cos(V)) * np.cos(U)
    Y = (R + r * np.cos(V)) * np.sin(U)
    Z = r * np.sin(V)

    # Calculate normals
    # Normals for torus at (u,v) is vector from center of tube circle to point:
    Nx = np.cos(V) * np.cos(U)
    Ny = np.cos(V) * np.sin(U)
    Nz = np.sin(V)
    normals = np.stack((Nx, Ny, Nz), axis=2)

    return X, Y, Z, normals

In [3]:
def plot_torus(X, Y, Z, title=None, noise=None):
    if noise is not None:
        # Create a figure with two subplots: 3D torus and 2D noise side by side
        fig = plt.figure(figsize=(12, 6))
        ax3d = fig.add_subplot(1, 2, 1, projection='3d')
        surf = ax3d.plot_surface(
            X, Y, Z,
            rstride=1, cstride=1,
            facecolors=cm.viridis((Z - Z.min()) / (Z.max() - Z.min())),
            linewidth=0, antialiased=False
        )
        ax3d.set_box_aspect([1, 1, 1])
        ax3d.set_xlim(-5, 5)
        ax3d.set_ylim(-5, 5)
        ax3d.set_zlim(-3, 3)
        if title:
            ax3d.set_title(title)

        ax2d = fig.add_subplot(1, 2, 2)
        im = ax2d.imshow(noise, cmap='gray', aspect='auto')
        ax2d.set_title("Noise Pattern")
        ax2d.axis('off')

        plt.tight_layout()
        plt.show()
    else:
        fig = plt.figure(figsize=(8, 6))
        ax = fig.add_subplot(111, projection='3d')
        ax.plot_surface(
            X, Y, Z,
            rstride=1, cstride=1,
            facecolors=cm.viridis((Z - Z.min()) / (Z.max() - Z.min())),
            linewidth=0, antialiased=False
        )
        ax.set_box_aspect([1, 1, 1])
        ax.set_xlim(-5, 5)
        ax.set_ylim(-5, 5)
        ax.set_zlim(-3, 3)
        if title:
            ax.set_title(title)
        plt.tight_layout()
        plt.show()

In [4]:
def base_torus(n_major=100, n_minor=30):
    X, Y, Z, _= generate_base_torus(n_major=n_major, n_minor=n_minor)
    plot_torus(X, Y, Z, f"Base Torus (n_major={n_major}, n_minor={n_minor})")

interactive(base_torus,
            n_major=IntSlider(min=10, max=200, step=10, value=100, description="Major divisions"),
            n_minor=IntSlider(min=5, max=100, step=5, value=30, description="Minor divisions")
           )

interactive(children=(IntSlider(value=100, description='Major divisions', max=200, min=10, step=10), IntSlider…

In [5]:
def random_noise(shape, seed=None):
    if not seed:
        seed = np.random.randint(1, 1000000)
    noise = np.random.rand(shape[0], shape[1])
    return noise

In [6]:
def perlin_noise(shape, octaves=1, seed=None):
    """
    Generates a smooth noise array of given shape using perlin_noise library    .
    """
    if not seed:
        seed = np.random.randint(1, 1000000)
    noise_generator = PerlinNoise(octaves=octaves, seed=seed)
    noise = np.array([[noise_generator([i / shape[0], j / shape[1]]) for j in range(shape[1])] for i in range(shape[0])])
    return noise

In [7]:
def worley_noise(shape, seed=None):
    if not seed:
        seed = np.random.randint(1, 1000000)
    shape = (int(shape[0]/10), int(shape[1]/10))
    w, c = worley(shape, dens=10, seed=seed)
    w = w[0].T
    return w

In [8]:
def star_noise(shape, seed=None):
    # Make rectangular grid
    x, y = np.arange(shape[0]), np.arange(shape[1])
    x, y = np.meshgrid(x, y, indexing="ij")

    # Generate noise: random displacements r at random angles phi
    np.random.seed(0)
    phi = np.random.uniform(0, 2 * np.pi, x.shape)
    r = np.random.uniform(0, 0.5, x.shape)

    # Shrink star size to keep it inside its cell.
    # Alse, we want more small stars - for the background effect.
    # To do that we rescale displacements: r -> 1/2 - 0.001 / r.
    r = np.clip(0.5 - 1e-3 / r, 0, None)
    size = 200 * (0.5 - r) - 0.4

    return size



In [9]:
# ## Torus with Random Noise
# Create a random noise pattern.

def torus_random_noise(n_major=100, n_minor=30, noise_strength=0.3):
    X, Y, Z, normals = generate_base_torus(n_major=n_major, n_minor=n_minor)

    # Generate noise based on param coords (U,V)
    noise = random_noise((n_minor, n_major))
    #noise = (noise - 0.5) * 2  # center noise around 0

    # Displace points along normals
    Xd = X + normals[:,:,0] * noise_strength * noise
    Yd = Y + normals[:,:,1] * noise_strength * noise
    Zd = Z + normals[:,:,2] * noise_strength * noise

    plot_torus(Xd, Yd, Zd,
               f"Torus with Random Noise\n, strength={noise_strength}", noise)
    

interactive(torus_random_noise,
            n_major=IntSlider(min=50, max=150, step=10, value=100, description="Major divisions"),
            n_minor=IntSlider(min=10, max=50, step=5, value=30, description="Minor divisions"),
            noise_strength=FloatSlider(min=0, max=1, step=0.05, value=0.3, description="Noise strength"),
           )

interactive(children=(IntSlider(value=100, description='Major divisions', max=150, min=50, step=10), IntSlider…

In [10]:
# ## Torus with Perlin Noise
# Create a smooth noise pattern.

def torus_perlin_noise(n_major=100, n_minor=30, noise_strength=0.3, octaves=3):
    X, Y, Z, normals = generate_base_torus(n_major=n_major, n_minor=n_minor)

    # Generate noise based on param coords (U,V)
    noise = perlin_noise((n_minor, n_major), octaves=octaves)
    #noise = (noise - 0.5) * 2  # center noise around 0

    # Displace points along normals
    Xd = X + normals[:,:,0] * noise_strength * noise
    Yd = Y + normals[:,:,1] * noise_strength * noise
    Zd = Z + normals[:,:,2] * noise_strength * noise

    plot_torus(Xd, Yd, Zd,
               f"Torus with Perlin Noise\n, strength={noise_strength}, octaves={octaves}", noise)
    

interactive(torus_perlin_noise,
            n_major=IntSlider(min=50, max=150, step=10, value=100, description="Major divisions"),
            n_minor=IntSlider(min=10, max=50, step=5, value=30, description="Minor divisions"),
            noise_scale=FloatSlider(min=1, max=20, step=0.5, value=5, description="Noise scale"),
            noise_strength=FloatSlider(min=0, max=1, step=0.05, value=0.3, description="Noise strength"),
            octaves=IntSlider(min=1, max=20, step=1, value=10, description="Octaves"),
           )

interactive(children=(IntSlider(value=100, description='Major divisions', max=150, min=50, step=10), IntSlider…

In [11]:
# ## Torus with Worley Noise
# Create a soft cellular propagation pattern.

def torus_worley_noise(n_major=100, n_minor=30, noise_strength=0.3):
    X, Y, Z, normals = generate_base_torus(n_major=n_major, n_minor=n_minor)

    noise = worley_noise((n_major, n_minor))

    # Displace points along normals
    Xd = X + normals[:,:,0] * noise_strength * noise
    Yd = Y + normals[:,:,1] * noise_strength * noise
    Zd = Z + normals[:,:,2] * noise_strength * noise

    plot_torus(Xd, Yd, Zd,
               f"Torus with Worley Noise\n, strength={noise_strength}", noise)
    

interactive(torus_worley_noise,
            n_major=IntSlider(min=50, max=150, step=10, value=100, description="Major divisions"),
            n_minor=IntSlider(min=10, max=50, step=5, value=30, description="Minor divisions"),
            noise_strength=FloatSlider(min=0, max=1, step=0.05, value=0.3, description="Noise strength"),
            shape=IntSlider(min=2, max=20, step=1, value=4, description="Noise Shape"),
            density=IntSlider(min=1, max=20, step=1, value=25, description="NoiseDensity"),
           )

interactive(children=(IntSlider(value=100, description='Major divisions', max=150, min=50, step=10), IntSlider…

In [12]:
# ## Torus with Star-Like Noise
# Create a star-like propagation pattern.

def torus_star_noise(n_major=100, n_minor=30, noise_strength=0.3): 
    X, Y, Z, normals = generate_base_torus(n_major=n_major, n_minor=n_minor)
    
    noise = star_noise((n_minor, n_major))
    noise = 0.1 * noise

    # Displace points along normals
    Xd = X + normals[:,:,0] * noise_strength * noise
    Yd = Y + normals[:,:,1] * noise_strength * noise
    Zd = Z + normals[:,:,2] * noise_strength * noise

    plot_torus(Xd, Yd, Zd,
               f"Torus with Star-Like Noise,\n strength={noise_strength}", noise)
    

interactive(torus_star_noise,
            n_major=IntSlider(min=50, max=150, step=10, value=100, description="Major divisions"),
            n_minor=IntSlider(min=10, max=50, step=5, value=30, description="Minor divisions"),
            noise_strength=FloatSlider(min=0, max=1, step=0.05, value=0.3, description="Noise strength"),
            shape=IntSlider(min=2, max=20, step=1, value=4, description="Noise Shape"),
            density=IntSlider(min=1, max=20, step=1, value=25, description="NoiseDensity"),
           )

interactive(children=(IntSlider(value=100, description='Major divisions', max=150, min=50, step=10), IntSlider…

In [13]:
# Torus with Cross-section Polar Modulation
# Modify the cross-section radius with a function in polar coordinates.

def torus_cross_section_polar_mod(n_major=100, n_minor=30, modulation_amplitude=0.3, modulation_frequency=5):
    R = 3
    r = 1
    u = np.linspace(0, 2*np.pi, n_major, endpoint=True)
    v = np.linspace(0, 2*np.pi, n_minor, endpoint=True)
    U, V = np.meshgrid(u, v)

    # Modulate minor radius as function of angle along minor circle
    mod = 1 + modulation_amplitude * np.sin(modulation_frequency * V)

    X = (R + r * mod * np.cos(V)) * np.cos(U)
    Y = (R + r * mod * np.cos(V)) * np.sin(U)
    Z = r * mod * np.sin(V)

    plot_torus(X, Y, Z,
               f"Torus with cross-section polar modulation\nAmplitude={modulation_amplitude}, Frequency={modulation_frequency}")

interactive(torus_cross_section_polar_mod,
            n_major=IntSlider(min=50, max=150, step=10, value=100, description="Major divisions"),
            n_minor=IntSlider(min=10, max=50, step=5, value=30, description="Minor divisions"),
            modulation_amplitude=FloatSlider(min=0, max=1, step=0.05, value=0.3, description="Amplitude"),
            modulation_frequency=IntSlider(min=1, max=20, step=1, value=5, description="Frequency"),
           )

interactive(children=(IntSlider(value=100, description='Major divisions', max=150, min=50, step=10), IntSlider…

In [14]:
# Torus with Möbius-like Twist
# Introduce a twist that varies with position along the major ring, creating a Möbius-like effect.

def torus_mobius_twist(n_major=100, n_minor=30, twist_strength=1.0):
    R = 3
    r = 1
    u = np.linspace(0, 2*np.pi, n_major, endpoint=True)
    v = np.linspace(0, 2*np.pi, n_minor, endpoint=True)
    U, V = np.meshgrid(u, v)

    # Start with base torus coordinates
    X_base = (R + r * np.cos(V)) * np.cos(U)
    Y_base = (R + r * np.cos(V)) * np.sin(U)
    Z_base = r * np.sin(V)

    # Create twist that varies with position along the major ring
    # Points on opposite sides of the torus twist in opposite directions
    twist_angle = twist_strength * np.sin(U)  # sin(U) changes sign halfway around
    
    # Apply the twist: rotate points around the local normal direction
    # This creates the Möbius-like effect where one side twists one way, the other side twists the opposite way
    cos_t = np.cos(twist_angle)
    sin_t = np.sin(twist_angle)
    
    # Twist the cross-section at each point along the major ring
    # This rotates the minor circle around the tangent direction
    X_twisted = X_base
    Y_twisted = Y_base * cos_t - Z_base * sin_t
    Z_twisted = Y_base * sin_t + Z_base * cos_t

    plot_torus(X_twisted, Y_twisted, Z_twisted,
               f"Torus with Möbius-like Twist\nStrength={twist_strength}")

interactive(torus_mobius_twist,
            n_major=IntSlider(min=50, max=150, step=10, value=100, description="Major divisions"),
            n_minor=IntSlider(min=10, max=50, step=5, value=30, description="Minor divisions"),
            twist_strength=FloatSlider(min=0, max=2, step=0.1, value=1.0, description="Twist Strength"),
           )

interactive(children=(IntSlider(value=100, description='Major divisions', max=150, min=50, step=10), IntSlider…

In [15]:
# Torus with Helical Warp
# Introduce a progressive rotation of the cross-section along the major ring, creating a spiral appearance.

def torus_helical_warp(n_major=100, n_minor=30, warp_strength=1.0):
    R = 3
    r = 1
    u = np.linspace(0, 2*np.pi, n_major, endpoint=True)
    v = np.linspace(0, 2*np.pi, n_minor, endpoint=True)
    U, V = np.meshgrid(u, v)

    # Start with base torus coordinates
    X_base = (R + r * np.cos(V)) * np.cos(U)
    Y_base = (R + r * np.cos(V)) * np.sin(U)
    Z_base = r * np.sin(V)

    # Create progressive helical warp that varies with position along the major ring
    # This creates a spiral effect where the twist increases as you go around the torus
    warp_angle = warp_strength * U  # Progressive twist from 0 to warp_strength * 2π
    
    # Apply the helical warp: rotate points around the local tangent direction
    # This creates a progressive spiral deformation
    cos_t = np.cos(warp_angle)
    sin_t = np.sin(warp_angle)
    
    # Helical warp: progressively twist the cross-section as you go around the major ring
    X_warped = X_base
    Y_warped = Y_base * cos_t - Z_base * sin_t
    Z_warped = Y_base * sin_t + Z_base * cos_t

    plot_torus(X_warped, Y_warped, Z_warped,
               f"Torus with Helical-Like Warp\nStrength={warp_strength}")

interactive(torus_helical_warp,
            n_major=IntSlider(min=50, max=150, step=10, value=100, description="Major divisions"),
            n_minor=IntSlider(min=10, max=50, step=5, value=30, description="Minor divisions"),
            warp_strength=FloatSlider(min=0, max=3, step=0.1, value=1.0, description="Warp Strength"),
           )

interactive(children=(IntSlider(value=100, description='Major divisions', max=150, min=50, step=10), IntSlider…

In [16]:
# Torus with Bend Deformation
# Curve the whole torus into an arc or S-shape.

# Alternative: More complex S-deformation with multiple curves
def torus_saddle_deformation(n_major=100, n_minor=30, s_strength=1.0):
    R = 3
    r = 1
    u = np.linspace(0, 2*np.pi, n_major, endpoint=True)
    v = np.linspace(0, 2*np.pi, n_minor, endpoint=True)
    U, V = np.meshgrid(u, v)

    # Start with base torus coordinates
    X_base = (R + r * np.cos(V)) * np.cos(U)
    Y_base = (R + r * np.cos(V)) * np.sin(U)
    Z_base = r * np.sin(V)

    # Create more complex S-curve with multiple inflection points
    s_bend_y = s_strength * (np.sin(2 * U) + 0.3 * np.sin(4 * U))
    s_bend_z = s_strength * (np.cos(2 * U) + 0.3 * np.cos(4 * U))
    
    X_s = X_base
    Y_s = Y_base + s_bend_y
    Z_s = Z_base + s_bend_z

    plot_torus(X_s, Y_s, Z_s,
               f"Torus with Saddle-Like Deformation\nStrength={s_strength}")

interactive(torus_saddle_deformation,
            n_major=IntSlider(min=50, max=150, step=10, value=100, description="Major divisions"),
            n_minor=IntSlider(min=10, max=50, step=5, value=30, description="Minor divisions"),
            bend_strength=FloatSlider(min=0, max=2, step=0.05, value=0.5, description="Bend Strength"),
           )

interactive(children=(IntSlider(value=100, description='Major divisions', max=150, min=50, step=10), IntSlider…

In [17]:
# Torus with Smooth Gradient Scaling Along Major Radius (Bulging)
# Scale the minor radius smoothly along the major ring for bulging/asymmetry without abrupt cutoffs.

def torus_gradient_scaling(n_major=100, n_minor=30, scale_min=0.5, scale_max=1.5):
    R = 3
    r = 1
    u = np.linspace(0, 2*np.pi, n_major, endpoint=True)  # endpoint=False to avoid seam
    v = np.linspace(0, 2*np.pi, n_minor, endpoint=True)
    U, V = np.meshgrid(u, v)

    # Create smooth gradient that flows from both sides
    # Use cosine function to create smooth transition that wraps around
    # This creates a bulge that flows smoothly from both sides toward the center
    gradient_factor = (scale_max - scale_min) / 2
    center_scale = (scale_max + scale_min) / 2
    
    # Create scales array directly with correct shape (1, n_major)
    # This will broadcast correctly with (n_minor, n_major) arrays
    scales = center_scale + gradient_factor * np.cos(u).reshape(1, -1)

    X = (R + r * scales * np.cos(V)) * np.cos(U)
    Y = (R + r * scales * np.cos(V)) * np.sin(U)
    Z = r * scales * np.sin(V)

    plot_torus(X, Y, Z,
               f"Torus with Gradient Scaling\nScale min={scale_min}, max={scale_max}")

interactive(torus_gradient_scaling,
            n_major=IntSlider(min=50, max=150, step=10, value=100, description="Major divisions"),
            n_minor=IntSlider(min=10, max=50, step=5, value=30, description="Minor divisions"),
            scale_min=FloatSlider(min=0.1, max=1, step=0.05, value=0.5, description="Scale Min"),
            scale_max=FloatSlider(min=1, max=3, step=0.05, value=1.5, description="Scale Max"),
           )

interactive(children=(IntSlider(value=100, description='Major divisions', max=150, min=50, step=10), IntSlider…

In [18]:
# ## 8. Torus with Sine Wave Deformation Along Major Ring
# Create undulating thickness variations along the torus main ring using sine waves

# %%
def torus_sine_wave_deformation(n_major=100, n_minor=30, amplitude=0.5, frequency=3, phase=0):
    R = 3
    r = 1
    u = np.linspace(0, 2*np.pi, n_major, endpoint=True)
    v = np.linspace(0, 2*np.pi, n_minor, endpoint=True)
    U, V = np.meshgrid(u, v)

    # Create sine wave modulation of the minor radius
    # This makes the torus thicker/thinner along the major ring
    radius_modulation = 1 + amplitude * np.sin(frequency * U + phase)
    
    # Apply the modulation to create undulating thickness
    X = (R + r * radius_modulation * np.cos(V)) * np.cos(U)
    Y = (R + r * radius_modulation * np.sin(V)) * np.sin(U)
    Z = r * radius_modulation * np.sin(V)

    plot_torus(X, Y, Z,
               f"Torus with Sine Wave Deformation\nAmplitude={amplitude}, Frequency={frequency}, Phase={phase}")

interactive(torus_sine_wave_deformation,
            n_major=IntSlider(min=50, max=150, step=10, value=100, description="Major divisions"),
            n_minor=IntSlider(min=10, max=50, step=5, value=30, description="Minor divisions"),
            amplitude=FloatSlider(min=0, max=1, step=0.05, value=0.5, description="Amplitude"),
            frequency=IntSlider(min=1, max=8, step=1, value=3, description="Frequency"),
            phase=FloatSlider(min=0, max=2*np.pi, step=0.1, value=0, description="Phase"),
           )

interactive(children=(IntSlider(value=100, description='Major divisions', max=150, min=50, step=10), IntSlider…